In [1]:
import os

## Column List

In [2]:
cols = ["Datetime",
        "Search term",
        "Google URL",
        "Ad URL Website",
        "Vendor",
        "Position Num",
        "Position",
        "Result is consistent",
        "Page number",
        "Type of result",
        "Comments",
        "Ad Value",
        "Static File Path"]

## Get user agent

In [3]:
#the default user_agent_list composes chrome,I E,firefox,Mozilla,opera,netscape
#for more user agent strings,you can find it in http://www.useragentstring.com/pages/useragentstring.php

user_agent_list = [\
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"\
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",\
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",\
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",\
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",\
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",\
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",\
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",\
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",\
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",\
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",\
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",\
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",\
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",\
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",\
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",\
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",\
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
       ]

# Imports

In [5]:
from bs4 import BeautifulSoup
import csv
import os
import random
import re
import requests
import subprocess
import urllib

### Advertisement class

In [453]:
class advertiz:
    def __init__(self, name):
        self.product_name = name
        
    def to_string(self):
        print("Ad Product Name   : %s" % self.product_name)
        print("Ad Product URL    : %s" % self.product_url)
        print("Ad Product Price  : %s" % self.price)
        print("Ad Product Vendor : %s" % self.vendor)
        #print("Ad file location  : %s" % self.filename)
        print()
        
    def get_random_filename(self):
        vendor = re.sub('[^0-9a-zA-Z]+', '_', self.vendor)
        self.filename = "../data/" + vendor + str(random.randint(1,100000)) + ".pdf"
        self.filename = os.path.abspath(self.filename)
        
    def convert_url_to_pdf(self):    
        self.get_random_filename()
        subprocess.run(["wkhtmltopdf", "%s" % self.product_url, self.filename])

### Ad results class

In [6]:
class AdResult:
    def __init__(self, keyword):
        self.keyword = keyword
        self.address = "http://www.google.com/search?q=%s&num=100&hl=en&start=0" % (urllib.parse.quote_plus(keyword))
        self.user    = self.process_request()
        self.request = urllib.request.Request(self.address, None, {'User-Agent': self.user})
        self.ads     = []
        # call search
        self.get_google_search_result()
        self.parse_ads()
        
    def process_request(self):
        ua = random.choice(user_agent_list)
        return ua
    
    def get_google_search_result(self):
        self.urlfile = urllib.request.urlopen(self.request)
        self.page    = self.urlfile.read()
        self.soup    = BeautifulSoup(self.page,'html.parser')
        
    def to_string(self):
        print("Keyword : %s" % self.keyword)
        print("Address : %s" % self.address)
        print("Title   : %s" % self.soup.title.string)
        
    def parse_ads(self):
        #get top ads
        try:
            self.top_ads      = self.soup.find(class_="top-pla-group-inner")
            self.top_ads_list = self.top_ads.find_all(class_="mnr-c pla-unit")
            self.parse_top_ads()
        except:
            print("Unable to parse top_ads\n")
            
        #get right ads
        #try:
           # self.right_ads = self.soup.find(id="rhs_block")
           # print(self.right_ads.prettify())
           # self.parse_right_ads()
        #except:
           # print("Unable to parse right_ads\n")
        
    #todo - all the ads
      
    def parse_right_ads(self):
        ad_data = self.right_ads.find(class_="_Ei rhsg4")
        self.right_ad_list = self.right_ads.find(class_="_Dw")
        for item in self.top_right_list:
            #create ad object
            ad              = advertiz(ad_data.get_text())
            ad.product_url  = ad_data['href']
           # ad.price        = item.span.find('span' ,attrs={class_="_kH"}).get_text()
            ad.vendor       = item.find(class_="rhsl4").get_text()
            ad.convert_url_to_pdf()
            self.ads.append(ad)
            
    def parse_top_ads(self):
        for item in self.top_ads_list:
            ad_data         = item.find(class_="_Z5 _BHp").find(class_="plantl pla-unit-title-link")
            #create ad object
            ad              = advertiz(ad_data.find('span', attrs={'class': '_GAv'}).text)
            ad.product_url  = ad_data['href']
            ad.price        = item.find(class_="_Z5 _BHp").find(class_="_QD _pvi").get_text()
            ad.vendor       = item.find(class_="_Z5 _BHp").find(class_="_mC").get_text()
            ad.convert_url_to_pdf()
            self.ads.append(ad)
    
    def convert_to_csv(self):
        with open("../data/Ads.csv", 'w') as myfile:
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerow(cols)
            for ad in self.ads:
                row = [datetime.datetime.now(),
                       self.keyword,
                       self.address, 
                       ad.product_url,
                       ad.vendor,
                       "NA",
                       "top ad", 
                       "NA", 
                       "1",
                       "Sponsored Ad", 
                       "NA",
                        ad.price,
                        ad.filename]
                wr.writerow(row)
            
        
            

In [7]:
ad_result = AdResult("Graco DreamGlider Gliding Swing & Sleeper Baby Swing, percy")
ad_result.to_string()


Unable to parse top_ads

Keyword : Graco DreamGlider Gliding Swing & Sleeper Baby Swing, percy
Address : http://www.google.com/search?q=Graco+DreamGlider+Gliding+Swing+%26+Sleeper+Baby+Swing%2C+percy&num=100&hl=en&start=0
Title   : Graco DreamGlider Gliding Swing & Sleeper Baby Swing, percy - Google Search


In [460]:
for ad in ad_result.ads:
    ad.to_string()
    


In [461]:
import datetime
ad_result.convert_to_csv()
